In [ ]:
import av
import torch
import numpy as np
import cv2
import os

from transformers import AutoProcessor, AutoModel
np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


for path in os.listdir("/srv/home/ahmadi/gitrepos/video-ir/scene_mp4"):
    path2 = os.path.join("/srv/home/ahmadi/gitrepos/video-ir/scene_mp4", path)
    print(path2)


    cap = cv2.VideoCapture(path2)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 

    print(n_frames)


    container = av.open(path2)
    indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)


        
    video = read_video_pyav(container, indices)

    processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
    model = AutoModel.from_pretrained("microsoft/xclip-base-patch32")
    inputs = processor(videos=list(video), return_tensors="pt")

    video_features = model.get_video_features(**inputs)

    # Save to file
    torch.save(video_features, '/srv/home/ahmadi/gitrepos/video-ir/vid_embedding/T-'+path+'.pt')


